In [25]:
import scipy.io
import pandas as pd
import h5py
import numpy as np

### Export

In [56]:
filepaths = []
subject = 1
session = 1
missing = 0
sub_total = 24 #24 is the default amount
master_df = pd.DataFrame(columns=['onset', 'duration', 'trial_type', 'response_time', 'sample', 'value','subject_id','session'])

# Loop through 24 subject and their 2 sessions to concatenation all data into single datafram: master_df.
# List files not found and count total sessions concatenated.
while subject <=sub_total:
    if subject <10:
        sub = '00' + str(subject)
    else:
        sub = '0' + str(subject)
        
    while session <=2: #there are max 2 sessions per subject
        ses = '0' + str(session)
        filepath = f'../ds001787-1.0.2/sub-{sub}/ses-{ses}/eeg/sub-{sub}_ses-{ses}_task-meditation_events.tsv'
        try:
            eeg_df = pd.read_csv(filepath, sep='\t', header=0)
        except:
            print(filepath.split('/')[5],"does not exist.")
            missing += 1
        eeg_df['subject_id'] = [sub] * len(eeg_df)
        eeg_df['session'] = [ses] *len(eeg_df)
#         print(eeg_df.head(25))
        master_df = pd.concat([master_df,eeg_df])
        session+=1
    subject += 1
    session = 1
total_ses = sub_total*2 - missing
print(f"Finished concatenating {total_ses} sessions.")

sub-008_ses-02_task-meditation_events.tsv does not exist.
sub-012_ses-02_task-meditation_events.tsv does not exist.
sub-013_ses-02_task-meditation_events.tsv does not exist.
sub-014_ses-02_task-meditation_events.tsv does not exist.
sub-015_ses-02_task-meditation_events.tsv does not exist.
sub-019_ses-02_task-meditation_events.tsv does not exist.
sub-020_ses-02_task-meditation_events.tsv does not exist.
sub-021_ses-02_task-meditation_events.tsv does not exist.
sub-024_ses-02_task-meditation_events.tsv does not exist.
Finished concatenating 39 sessions.


### Transform

In [57]:
# Reorder columns and check if duration and response_time columns are entirely NaN. Drop if so.
master_df = master_df[['subject_id','session','onset','duration','trial_type','response_time','sample','value']]
if master_df.duration.isnull().all():
    master_df.drop(columns=['duration'])
if master_df.response_time.isnull().all():
    master_df.drop(columns=['response_time'])
print(master_df.head(10))

  subject_id session       onset  duration trial_type  response_time   sample  \
0        001      01   71.386719       NaN   stimulus            NaN  18275.0   
1        001      01   75.730469       NaN   response            NaN  19387.0   
2        001      01   79.773438       NaN   response            NaN  20422.0   
3        001      01  125.609375       NaN   stimulus            NaN  32156.0   
4        001      01  179.800781       NaN   stimulus            NaN  46029.0   
5        001      01  183.097656       NaN   response            NaN  46873.0   
6        001      01  185.632812       NaN   response            NaN  47522.0   
7        001      01  284.859375       NaN   stimulus            NaN  72924.0   
8        001      01  287.757812       NaN   response            NaN  73666.0   
9        001      01  289.960938       NaN   response            NaN  74230.0   

  value  
0   128  
1     2  
2     2  
3   128  
4   128  
5     2  
6     4  
7   128  
8     2  
9     2 

In [58]:
master_df.columns

Index(['subject_id', 'session', 'onset', 'duration', 'trial_type',
       'response_time', 'sample', 'value'],
      dtype='object')

In [ ]:
# value = button press code? stim = 128, response = 2, 4

### Create database connection

In [ ]:
connection_string = "postgres:raw123@localhost:5432/eeg_db"
engine = create_engine(f'postgresql://{connection_string}')

In [ ]:
# Confirm tables
engine.table_names()

### Load Database into PostGres

In [ ]:
master_df.to_sql(name='master_eeg', con=engine, if_exists='append', index=True)